In [1]:
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from tqdm import tqdm

import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from catboost import CatBoostClassifier

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import lightgbm as lgb

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
train = pd.read_csv('train_sent.csv')
test = pd.read_csv('test_sent.csv')

listings = pd.read_csv('all_listings.csv')

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (24,26,47,49,52,54,59,61,65,66,73,88,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,compound,neg,neu,pos
0,6,93,9.0,9.0,10.0,10.0,9.0,9.0,0.906302,0.013705,0.660295,0.326000
1,109,80,10.0,10.0,6.0,8.0,10.0,8.0,0.479400,0.014000,0.854000,0.132500
2,344,80,10.0,10.0,10.0,10.0,10.0,8.0,0.492650,0.000000,0.792000,0.208000
3,686,95,9.0,10.0,10.0,10.0,10.0,10.0,0.899540,0.010952,0.690776,0.298262
4,930,98,10.0,10.0,10.0,10.0,10.0,10.0,0.883906,0.019000,0.591687,0.389250


In [49]:
test['review_scores_rating'] = 90
test['review_scores_accuracy'] = 9
test['review_scores_cleanliness'] = 9
test['review_scores_checkin'] = 9
test['review_scores_communication'] = 9
test['review_scores_location'] = 9
test['review_scores_value'] = 9
test_compound = test['compound']
test_neg = test['neg']
test_neu = test['neu']
test_pos = test['pos']
test.drop(['compound', 'neg', 'neu', 'pos'], axis=1, inplace=True)
test['compound'] = test_compound
test['neg'] = test_neg
test['neu'] = test_neu
test['pos'] = test_pos

In [50]:
test.head()

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,compound,neg,neu,pos
0,177,90,9,9,9,9,9,9,0.789695,0.020895,0.655263,0.323816
1,590,90,9,9,9,9,9,9,0.887377,0.010110,0.679164,0.308788
2,958,90,9,9,9,9,9,9,0.821159,0.014551,0.691268,0.286315
3,1788,90,9,9,9,9,9,9,0.886797,0.002567,0.695867,0.301567
4,2384,90,9,9,9,9,9,9,0.909629,0.011200,0.706920,0.281907


In [8]:
listings.head().T

,0,1,2,3,4
access,NaN,NaN,NaN,NaN,NaN
accommodates,2,4,1,6,6
amenities,"{""Wireless Internet"",""Air Conditioning"",Kitche...","{TV,""Cable TV"",Internet,""Wireless Internet"",""A...","{TV,""Wireless Internet"",""Air Conditioning"",Kit...","{Internet,""Wireless Internet"",""Air Conditionin...","{Internet,""Wireless Internet"",""Free Parking on..."
availability_30,0,17,29,28,25
availability_365,326,228,364,361,353
availability_60,21,36,59,56,49
availability_90,51,57,89,86,78
bathrooms,1,1,1,2.5,2.5
bed_type,Real Bed,Real Bed,Real Bed,Real Bed,Real Bed
bedrooms,1,1,1,1,1


In [9]:
listings.rename(columns={'id':'listing_id'}, inplace=True)

In [10]:
train = train.merge(listings, on='listing_id', how='left')

In [12]:
train.head().T

,0,1,2,3,4
listing_id,6,109,344,686,930
review_scores_rating,93,80,80,95,98
review_scores_accuracy,9,10,10,9,10
review_scores_cleanliness,9,10,10,10,10
review_scores_checkin,10,6,10,10,10
review_scores_communication,10,8,10,10,10
review_scores_location,9,10,10,10,10
review_scores_value,9,8,8,10,10
compound,0.906302,0.4794,0.49265,0.89954,0.883906
neg,0.0137045,0.014,0,0.0109524,0.019


In [51]:
test = test.merge(listings, on='listing_id', how='left')

In [47]:
train['transit'][3]

"Getting around Denver is easy from here without being right in the middle of the traffic, noise, and pollution. Central Denver's best amenities are all in walking distance of my home, but if you'd rather not walk, there is a bike-share rack only one block away.   If bicycling doesn't appeal, there are three RTD Bus routes that have stops within 2 blocks of my home: check out the #20, the #28, and the #32 for ease of access into Downtown, from Downtown, and to and from Denver's City Park and its great Museum of Nature and Science and the Denver Zoo. The #15 Bus along Colfax Avenue is only 4 blocks away, and can take guests along East Colfax to the major medical campuses on its route, and near the Denver Botanic Gardens. My home is 8 short blocks from access to Denver's light rail.  For guests arriving by car, I have a parking permit which - when properly displayed - allows guests to park on the street for up to 72 hours without having to move the car.  There is a $20 deposit for the us

In [24]:
sid = SentimentIntensityAnalyzer()

In [46]:
sid.polarity_scores(train['transit'][3])

{'compound': 0.3919, 'neg': 0.066, 'neu': 0.866, 'pos': 0.069}

In [52]:
test.head().T

,0,1,2,3,4
listing_id,177,590,958,1788,2384
review_scores_rating,90,90,90,90,90
review_scores_accuracy,9,9,9,9,9
review_scores_cleanliness,9,9,9,9,9
review_scores_checkin,9,9,9,9,9
review_scores_communication,9,9,9,9,9
review_scores_location,9,9,9,9,9
review_scores_value,9,9,9,9,9
compound,0.789695,0.887377,0.821159,0.886797,0.909629
neg,0.0208947,0.01011,0.0145512,0.00256667,0.0112


In [53]:
train_test = pd.concat([train, test])

In [55]:
train_test.shape

(112024, 104)

In [57]:
train_test['access'].fillna('none', inplace=True)

In [58]:
tqdm.pandas()
train_test['access_sentiment'] = train_test['access'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [00:56<00:00, 1985.36it/s]


In [59]:
train_test['access_compound'] = train_test['access_sentiment'].progress_apply(lambda x: x['compound'])
train_test['access_neg'] = train_test['access_sentiment'].progress_apply(lambda x: x['neg'])
train_test['access_neu'] = train_test['access_sentiment'].progress_apply(lambda x: x['neu'])
train_test['access_pos'] = train_test['access_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 497884.66it/s]


In [60]:
train_test.drop('access', axis=1, inplace=True)

In [463]:
train_test.drop('access_sentiment', axis=1, inplace=True)

In [82]:
test['accommodates'].value_counts()

2     16838
4      7541
3      4143
1      3530
6      3499
5      2046
8      1316
10      720
7       584
12      414
9       185
16      142
11       63
14       61
15       26
13       21
20        1
Name: accommodates, dtype: int64

In [73]:
#temp = pd.DataFrame()
temp = train.groupby('accommodates')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['accommodates'] + [('mean_accommodates_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='accommodates', how='left')

In [84]:
train_test['mean_accommodates_review_scores_rating'].fillna(train_test['mean_accommodates_review_scores_rating'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_accuracy'].fillna(train_test['mean_accommodates_review_scores_accuracy'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_cleanliness'].fillna(train_test['mean_accommodates_review_scores_cleanliness'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_checkin'].fillna(train_test['mean_accommodates_review_scores_checkin'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_communication'].fillna(train_test['mean_accommodates_review_scores_communication'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_location'].fillna(train_test['mean_accommodates_review_scores_location'].mean(), inplace=True)
train_test['mean_accommodates_review_scores_value'].fillna(train_test['mean_accommodates_review_scores_value'].mean(), inplace=True)

In [94]:
train_test['number_of_amenities'] = train_test['amenities'].apply(lambda x: len(x.split(',')))

In [95]:
train_test.drop('amenities', axis=1, inplace=True)

In [99]:
train_test['availability_90'].isnull().sum()

0

In [104]:
train_test['bed_type'].value_counts()

Real Bed         109193
Futon              1118
Pull-out Sofa       816
Airbed              602
Couch               295
Name: bed_type, dtype: int64

In [105]:
temp = train.groupby('bed_type')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['bed_type'] + [('mean_bed_type_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='bed_type', how='left')
train_test.drop('bed_type', axis=1, inplace=True)

In [123]:
train_test.head().T

,0,1,2,3,4
listing_id,6,109,344,686,930
review_scores_rating,93,80,80,95,98
review_scores_accuracy,9,10,10,9,10
review_scores_cleanliness,9,10,10,10,10
review_scores_checkin,10,6,10,10,10
review_scores_communication,10,8,10,10,10
review_scores_location,9,10,10,10,10
review_scores_value,9,8,8,10,10
compound,0.906302,0.4794,0.49265,0.89954,0.883906
neg,0.0137045,0.014,0,0.0109524,0.019


In [108]:
train_test['calculated_host_listings_count'].fillna(1, inplace=True)

In [111]:
train_test['bathrooms'].fillna(1, inplace=True)

In [113]:
train_test['bedrooms'].fillna(1, inplace=True)

In [115]:
train_test['beds'].fillna(1, inplace=True)

In [116]:
train_test.drop(['calendar_last_scraped', 'calendar_updated'], axis=1, inplace=True)

In [119]:
temp = train.groupby('cancellation_policy')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['cancellation_policy'] + [('mean_cancellation_policy_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='cancellation_policy', how='left')
train_test.drop('cancellation_policy', axis=1, inplace=True)

In [122]:
temp = train.groupby('city')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['city'] + [('mean_city_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='city', how='left')
train_test.drop('city', axis=1, inplace=True)

In [130]:
train_test['cleaning_fee'].value_counts()

0            20581
$50.00        9431
$100.00       7085
$25.00        6068
$75.00        5274
$20.00        5255
$30.00        4702
$40.00        3911
$60.00        3881
$10.00        3785
$15.00        3550
$150.00       3494
$35.00        3022
$80.00        2842
$65.00        1920
$125.00       1911
$0.00         1893
$45.00        1860
$70.00        1483
$85.00        1330
$120.00       1304
$200.00       1167
$5.00         1164
$90.00        1134
$95.00        1064
$55.00         956
$99.00         751
$250.00        644
$175.00        561
$110.00        435
             ...  
$316.00          1
$579.00          1
$813.00          1
$136.00          1
$127.00          1
$197.00          1
$133.00          1
$286.00          1
$202.00          1
$258.00          1
$749.00          1
$625.00          1
$161.00          1
$201.00          1
$177.00          1
$575.00          1
$900.00          1
$346.00          1
$950.00          1
$474.00          1
$382.00          1
$342.00     

In [131]:
train_test['cleaning_fee'].fillna('0', inplace=True)
def cleaning_fee(raw):
    if raw == 0:
        return 0.0
    if len(raw)>0:
        if ',' in raw:
            raw = raw.replace(',', '')
        if raw[0] != '$':
            return float(raw)
        else:
            return float(raw[1:])
    else:
        return 0.0
train_test['cleaning_fee'] = train_test['cleaning_fee'].apply(cleaning_fee)

In [132]:
train_test.drop(['country', 'country_code'], axis=1, inplace=True)

In [140]:
train_test['description'].fillna('none', inplace=True)

In [138]:
sid.polarity_scores(train_test['description'][107])

{'compound': 0.9274, 'neg': 0.04, 'neu': 0.851, 'pos': 0.109}

In [141]:
tqdm.pandas()
train_test['description_sentiment'] = train_test['description'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [07:15<00:00, 257.43it/s]


In [142]:
train_test['description_compound'] = train_test['description_sentiment'].progress_apply(lambda x: x['compound'])
train_test['description_neg'] = train_test['description_sentiment'].progress_apply(lambda x: x['neg'])
train_test['description_neu'] = train_test['description_sentiment'].progress_apply(lambda x: x['neu'])
train_test['description_pos'] = train_test['description_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 315560.55it/s]


In [143]:
train_test.drop(['description', 'description_sentiment'], axis=1, inplace=True)

In [145]:
train_test.drop('experiences_offered', axis=1, inplace=True)

In [146]:
train_test['extra_people'] = train_test['extra_people'].apply(cleaning_fee)

In [155]:
train_test['first_review'].fillna('2017-01-01', inplace=True)
train_test['first_review'] = pd.to_datetime(train_test['first_review'])
train_test['Days_since_first_review'] = pd.to_datetime('2018-07-20') - train_test['first_review']
train_test['Days_since_first_review'] = train_test['Days_since_first_review'].apply(lambda x: int(str(x)[:-14]))

In [157]:
train_test.drop('first_review', axis=1, inplace=True)

In [159]:
train_test['guests_included'].value_counts()

1     69218
2     24535
4      8319
3      3242
6      2889
5      1364
8      1025
0       404
10      397
7       309
12      115
9        73
16       38
14       30
15       26
11       23
13       15
18        1
20        1
Name: guests_included, dtype: int64

In [161]:
train_test['has_availability'].fillna(0, inplace=True)
train_test['has_availability'].loc[train_test['has_availability'] == 't'] = 1

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [163]:
list(train_test.columns)

['listing_id',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'compound',
 'neg',
 'neu',
 'pos',
 'accommodates',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'cleaning_fee',
 'extra_people',
 'guests_included',
 'has_availability',
 'host_about',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_location',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_response_rate',
 'host_response_time',
 'host_since',
 'host_thumbnail_url',
 'host_total_listings_count',
 'host_url',
 'host_verifications',
 'house_rules',
 'instant_bookable',
 'interaction',
 'is_business_travel_ready',
 'is_location_exact',
 'jurisdiction_names',
 'last_rev

In [175]:
train_test['host_about'].fillna('none', inplace=True)

In [173]:
sid.polarity_scores(train_test['host_about'][175])

{'compound': 0.5267, 'neg': 0.051, 'neu': 0.847, 'pos': 0.102}

In [176]:
tqdm.pandas()
train_test['host_about_sentiment'] = train_test['host_about'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [02:25<00:00, 768.68it/s] 


In [177]:
train_test['host_about_compound'] = train_test['host_about_sentiment'].progress_apply(lambda x: x['compound'])
train_test['host_about_neg'] = train_test['host_about_sentiment'].progress_apply(lambda x: x['neg'])
train_test['host_about_neu'] = train_test['host_about_sentiment'].progress_apply(lambda x: x['neu'])
train_test['host_about_pos'] = train_test['host_about_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 304413.02it/s]


In [178]:
train_test.drop(['host_about', 'host_about_sentiment'], axis=1, inplace=True)

In [181]:
train_test['host_acceptance_rate'].fillna('100%', inplace=True)

def host_response_rate(raw):
    if len(raw)>0:
        if ',' in raw:
            raw = raw.replace(',', '')
        if raw[-1] != '%':
            return float(raw)
        else:
            return float(raw[:-1])
    else:
        return 0.0
    
train_test['host_acceptance_rate'] = train_test['host_acceptance_rate'].apply(host_response_rate)

In [186]:
train_test['host_has_profile_pic'].fillna('t', inplace=True)
train_test['host_has_profile_pic'].loc[train_test['host_has_profile_pic'] == 't'] = 1
train_test['host_has_profile_pic'].loc[train_test['host_has_profile_pic'] == 'f'] = 0

In [190]:
temp = train_test['host_id'].value_counts().reset_index()
temp.rename(columns={'index':'host_id', 'host_id':'number_of_listings_by_this_host'}, inplace=True)
train_test = train_test.merge(temp, on='host_id', how='left')

In [191]:
train_test.drop('host_id', axis=1, inplace=True)

In [194]:
train_test['host_identity_verified'].fillna('t', inplace=True)
train_test['host_identity_verified'].loc[train_test['host_identity_verified'] == 't'] = 1
train_test['host_identity_verified'].loc[train_test['host_identity_verified'] == 'f'] = 0

In [198]:
train_test['host_is_superhost'].fillna('f', inplace=True)
train_test['host_is_superhost'].loc[train_test['host_is_superhost'] == 't'] = 1
train_test['host_is_superhost'].loc[train_test['host_is_superhost'] == 'f'] = 0

In [201]:
train_test['host_listings_count'].fillna(1, inplace=True)

In [204]:
train_test.drop('host_location', axis=1, inplace=True)

In [207]:
train_test.drop('host_name', axis=1, inplace=True)

In [209]:
train_test.drop('host_neighbourhood', axis=1, inplace=True)

In [210]:
train_test.drop('host_picture_url', axis=1, inplace=True)

In [213]:
train_test['host_response_rate'].fillna('100%', inplace=True)
train_test['host_response_rate'] = train_test['host_response_rate'].apply(host_response_rate)

In [219]:
train_test['host_response_time'].fillna('within an hour', inplace=True)

In [220]:
temp = train.groupby('host_response_time')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['host_response_time'] + [('mean_host_response_time_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='host_response_time', how='left')
train_test.drop('host_response_time', axis=1, inplace=True)

In [224]:
train_test['host_since'].fillna('2017-01-01', inplace=True)
train_test['host_since'] = pd.to_datetime(train_test['host_since'])
train_test['host_since_number_of_days'] = pd.to_datetime('2018-07-01') - train_test['host_since']
train_test['host_since_number_of_days'] = train_test['host_since_number_of_days'].apply(lambda x: int(str(x)[:-14]))

In [226]:
train_test.drop('host_since', axis=1, inplace=True)

In [229]:
train_test.drop(['host_thumbnail_url', 'host_total_listings_count', 'host_url'], axis=1, inplace=True)

In [231]:
train_test['number_of_host_verifications'] = train_test['host_verifications'].apply(lambda x: len(x.split(',')))

In [232]:
train_test.drop('host_verifications', axis=1, inplace=True)

In [243]:
train_test['house_rules'].fillna('none', inplace=True)

In [244]:
tqdm.pandas()
train_test['house_rules_sentiment'] = train_test['house_rules'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [02:29<00:00, 748.79it/s] 


In [245]:
train_test['house_rules_compound'] = train_test['house_rules_sentiment'].progress_apply(lambda x: x['compound'])
train_test['house_rules_neg'] = train_test['house_rules_sentiment'].progress_apply(lambda x: x['neg'])
train_test['house_rules_neu'] = train_test['house_rules_sentiment'].progress_apply(lambda x: x['neu'])
train_test['house_rules_pos'] = train_test['house_rules_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 290971.44it/s]


In [246]:
train_test.drop(['house_rules', 'house_rules_sentiment'], axis=1, inplace=True)

In [247]:
list(train_test.columns)

['listing_id',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'compound',
 'neg',
 'neu',
 'pos',
 'accommodates',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'cleaning_fee',
 'extra_people',
 'guests_included',
 'has_availability',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_response_rate',
 'instant_bookable',
 'interaction',
 'is_business_travel_ready',
 'is_location_exact',
 'jurisdiction_names',
 'last_review',
 'last_scraped',
 'last_searched',
 'latitude',
 'license',
 'listing_url',
 'longitude',
 'market',
 'maximum_nights',
 'medium_url',
 'minimum_nights',
 'monthly_price',
 'name',
 'neighborhood_overview',
 'neighbourhood',
 'neighbourhood

In [250]:
train_test['instant_bookable'].loc[train_test['instant_bookable'] == 't'] = 1
train_test['instant_bookable'].loc[train_test['instant_bookable'] == 'f'] = 0

In [253]:
train_test['interaction'].fillna('none', inplace=True)

In [256]:
train_test['interaction'][10]

'We will have a list of Harlem restaurants and points of interest ready for you, as well as a subway map of NYC and pratical infos.'

In [257]:
sid.polarity_scores(train_test['interaction'][10])

{'compound': 0.765, 'neg': 0.0, 'neu': 0.734, 'pos': 0.266}

In [258]:
tqdm.pandas()
train_test['interaction'] = train_test['interaction'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [01:06<00:00, 1687.90it/s]


In [260]:
train_test['interaction_compound'] = train_test['interaction'].progress_apply(lambda x: x['compound'])
train_test['interaction_neg'] = train_test['interaction'].progress_apply(lambda x: x['neg'])
train_test['interaction_neu'] = train_test['interaction'].progress_apply(lambda x: x['neu'])
train_test['interactionpos'] = train_test['interaction'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 319156.63it/s]


In [262]:
train_test.drop('interaction', inplace=True, axis=1)

In [265]:
train_test['is_business_travel_ready'].fillna('f', inplace=True)
train_test['is_business_travel_ready'].loc[train_test['is_business_travel_ready'] == 'f'] = 0
train_test['is_business_travel_ready'].loc[train_test['is_business_travel_ready'] == 't'] = 1

In [268]:
train_test['is_location_exact'].loc[train_test['is_location_exact'] == 't'] = 1
train_test['is_location_exact'].loc[train_test['is_location_exact'] == 'f'] = 0

In [270]:
train_test.drop('jurisdiction_names', axis=1, inplace=True)

In [273]:
train_test['last_review'].fillna('2017-01-01', inplace=True)
train_test['last_review'] = pd.to_datetime(train_test['last_review'])
train_test['days_since_last_review'] = pd.to_datetime('2018-07-01') - train_test['last_review']
train_test['days_since_last_review'] = train_test['days_since_last_review'].apply(lambda x: int(str(x)[:-14]))

In [275]:
train_test.drop('last_review', axis=1, inplace=True)

In [278]:
train_test['last_scraped'] = pd.to_datetime(train_test['last_scraped'])
train_test['days_since_last_scraping'] = pd.to_datetime('2018-07-01') - train_test['last_scraped']
train_test['days_since_last_scraping'] = train_test['days_since_last_scraping'].apply(lambda x: int(str(x)[:-14]))

In [280]:
train_test.drop('last_scraped', axis=1, inplace=True)

In [283]:
train_test['last_searched'].fillna('2017-01-01', inplace=True)
train_test['last_searched'] = pd.to_datetime(train_test['last_searched'])
train_test['days_since_last_searching'] = pd.to_datetime('2018-07-01') - train_test['last_searched']
train_test['days_since_last_searching'] = train_test['days_since_last_searching'].apply(lambda x: int(str(x)[:-14]))

In [285]:
train_test.drop('last_searched', axis=1, inplace=True)

In [287]:
train_test.drop('license', axis=1, inplace=True)

In [288]:
train_test.drop(['listing_url', 'latitude', 'longitude'], axis=1, inplace=True)

In [301]:
temp = train.groupby('market')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['market'] + [('mean_market_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='market', how='left')
train_test.drop('market', axis=1, inplace=True)

In [302]:
train_test['mean_market_review_scores_rating'].fillna(train_test['mean_market_review_scores_rating'].mean(), inplace=True)
train_test['mean_market_review_scores_accuracy'].fillna(train_test['mean_market_review_scores_accuracy'].mean(), inplace=True)
train_test['mean_market_review_scores_cleanliness'].fillna(train_test['mean_market_review_scores_cleanliness'].mean(), inplace=True)
train_test['mean_market_review_scores_checkin'].fillna(train_test['mean_market_review_scores_checkin'].mean(), inplace=True)
train_test['mean_market_review_scores_communication'].fillna(train_test['mean_market_review_scores_communication'].mean(), inplace=True)
train_test['mean_market_review_scores_location'].fillna(train_test['mean_market_review_scores_location'].mean(), inplace=True)
train_test['mean_market_review_scores_value'].fillna(train_test['mean_market_review_scores_value'].mean(), inplace=True)

In [304]:
train_test['maximum_nights'].value_counts()

1125         63215
30            6773
14            4122
7             3553
365           3203
90            2537
60            2272
28            2131
29            2104
10            2002
5             1530
21            1527
31            1296
15            1262
180           1098
20             954
3              898
4              885
6              730
120            726
25             620
8              601
12             460
45             459
100            378
27             374
9              369
40             320
2              319
360            247
             ...  
3651365          1
1043             1
2323             1
146              1
3600             1
7000             1
1115             1
1180             1
1508             1
1064             1
104              1
1127             1
550              1
422              1
358              1
230              1
166              1
1061             1
1122             1
3100             1
866              1
993         

In [306]:
train_test.drop('medium_url', axis=1, inplace=True)

In [309]:
train_test['minimum_nights'].value_counts().describe()

count       91.000000
mean      1231.032967
std       6009.559040
min          1.000000
25%          1.000000
50%          4.000000
75%         44.000000
max      41238.000000
Name: minimum_nights, dtype: float64

In [314]:
train_test.drop('monthly_price', axis=1, inplace=True)

In [324]:
train_test['name'][17]

'Available! Perfect location! Entire apartment!'

In [325]:
sid.polarity_scores(train_test['name'][17])

{'compound': 0.6784, 'neg': 0.0, 'neu': 0.466, 'pos': 0.534}

In [328]:
train_test['name'].fillna('none', inplace=True)

In [329]:
tqdm.pandas()
train_test['name_sentiment'] = train_test['name'].progress_apply(lambda x: sid.polarity_scores(x))


100%|██████████| 112024/112024 [00:22<00:00, 4879.52it/s]

In [330]:
train_test['name_compound'] = train_test['name_sentiment'].progress_apply(lambda x: x['compound'])
train_test['name_neg'] = train_test['name_sentiment'].progress_apply(lambda x: x['neg'])
train_test['name_neu'] = train_test['name_sentiment'].progress_apply(lambda x: x['neu'])
train_test['name_pos'] = train_test['name_sentiment'].progress_apply(lambda x: x['pos'])


100%|██████████| 112024/112024 [00:00<00:00, 362537.20it/s]

In [331]:
train_test.drop(['name', 'name_sentiment'], axis=1, inplace=True)

In [332]:
list(train_test.columns)

['listing_id',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'compound',
 'neg',
 'neu',
 'pos',
 'accommodates',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'cleaning_fee',
 'extra_people',
 'guests_included',
 'has_availability',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_response_rate',
 'instant_bookable',
 'is_business_travel_ready',
 'is_location_exact',
 'maximum_nights',
 'minimum_nights',
 'neighborhood_overview',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'notes',
 'number_of_reviews',
 'picture_url',
 'price',
 'property_type',
 'region_id',
 'region_name',
 'region_parent_id',
 'require_guest_phone_veri

In [335]:
train_test['neighborhood_overview'].fillna('none', inplace=True)

In [336]:
tqdm.pandas()
train_test['neighborhood_overview_sentiment'] = train_test['neighborhood_overview'].progress_apply(lambda x: sid.polarity_scores(x))


 21%|██▏       | 23977/112024 [00:31<01:44, 839.90it/s]


 46%|████▋     | 52029/112024 [01:02<00:58, 1031.69it/s]


 71%|███████   | 79635/112024 [01:34<00:46, 689.37it/s]


 93%|█████████▎| 104510/112024 [02:05<00:09, 784.81it/s]


100%|██████████| 112024/112024 [02:13<00:00, 840.77it/s] 

In [337]:
train_test['neighborhood_overview_compound'] = train_test['neighborhood_overview_sentiment'].progress_apply(lambda x: x['compound'])
train_test['neighborhood_overview_neg'] = train_test['neighborhood_overview_sentiment'].progress_apply(lambda x: x['neg'])
train_test['neighborhood_overview_neu'] = train_test['neighborhood_overview_sentiment'].progress_apply(lambda x: x['neu'])
train_test['neighborhood_overview_pos'] = train_test['neighborhood_overview_sentiment'].progress_apply(lambda x: x['pos'])


100%|██████████| 112024/112024 [00:00<00:00, 348984.58it/s]

In [338]:
train_test.drop(['neighborhood_overview', 'neighborhood_overview_sentiment'], axis=1, inplace=True)

In [343]:
train_test.drop('neighbourhood', axis=1, inplace=True)

In [347]:
train_test.drop(['neighbourhood_cleansed', 'neighbourhood_group_cleansed'], axis=1, inplace=True)

In [350]:
train_test['notes'].fillna('none', inplace=True)

In [351]:
tqdm.pandas()
train_test['notes_sentiment'] = train_test['notes'].progress_apply(lambda x: sid.polarity_scores(x))


 28%|██▊       | 30830/112024 [00:31<01:12, 1114.33it/s]


 61%|██████    | 68058/112024 [01:03<00:29, 1476.71it/s]


 89%|████████▉ | 99574/112024 [01:35<00:07, 1559.26it/s]


100%|██████████| 112024/112024 [01:44<00:00, 1074.15it/s]

In [352]:
train_test['notes_compound'] = train_test['notes_sentiment'].progress_apply(lambda x: x['compound'])
train_test['notes_neg'] = train_test['notes_sentiment'].progress_apply(lambda x: x['neg'])
train_test['notes_neu'] = train_test['notes_sentiment'].progress_apply(lambda x: x['neu'])
train_test['notes_pos'] = train_test['notes_sentiment'].progress_apply(lambda x: x['pos'])


100%|██████████| 112024/112024 [00:00<00:00, 424333.05it/s]

In [354]:
train_test.drop(['notes', 'notes_sentiment'], axis=1, inplace=True)

In [356]:
train_test['number_of_reviews'].value_counts()

1      12057
2       8806
3       6630
4       5427
5       4656
6       3872
7       3384
8       3074
9       2705
10      2574
11      2242
12      2110
13      1958
14      1872
15      1796
16      1606
17      1490
18      1455
19      1359
20      1292
21      1211
22      1158
24      1088
23      1086
25       996
26       942
28       930
27       901
29       829
31       821
       ...  
404        1
530        1
466        1
464        1
406        1
400        1
399        1
398        1
395        1
331        1
522        1
342        1
471        1
355        1
413        1
417        1
480        1
735        1
415        1
605        1
477        1
412        1
472        1
348        1
347        1
730        1
474        1
473        1
409        1
0          1
Name: number_of_reviews, Length: 464, dtype: int64

In [358]:
train_test.drop('picture_url', axis=1, inplace=True)

In [361]:
train_test['price'] = train_test['price'].apply(cleaning_fee)

In [365]:
train_test['property_type'].fillna('Apartment', inplace=True)

In [374]:
temp = train.groupby('property_type')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['property_type'] + [('mean_property_type_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='property_type', how='left')
train_test.drop('property_type', axis=1, inplace=True)

In [375]:
train_test['mean_property_type_review_scores_rating'].fillna(train_test['mean_property_type_review_scores_rating'].mean(), inplace=True)
train_test['mean_property_type_review_scores_accuracy'].fillna(train_test['mean_property_type_review_scores_accuracy'].mean(), inplace=True)
train_test['mean_property_type_review_scores_cleanliness'].fillna(train_test['mean_property_type_review_scores_cleanliness'].mean(), inplace=True)
train_test['mean_property_type_review_scores_checkin'].fillna(train_test['mean_property_type_review_scores_checkin'].mean(), inplace=True)
train_test['mean_property_type_review_scores_communication'].fillna(train_test['mean_property_type_review_scores_communication'].mean(), inplace=True)
train_test['mean_property_type_review_scores_location'].fillna(train_test['mean_property_type_review_scores_location'].mean(), inplace=True)
train_test['mean_property_type_review_scores_value'].fillna(train_test['mean_property_type_review_scores_value'].mean(), inplace=True)

In [380]:
train_test.drop(['region_id', 'region_name', 'region_parent_id'], axis=1, inplace=True)

In [383]:
train_test['require_guest_phone_verification'].loc[train_test['require_guest_phone_verification'] == 'f'] = 0
train_test['require_guest_phone_verification'].loc[train_test['require_guest_phone_verification'] == 't'] = 1

In [386]:
train_test['require_guest_profile_picture'].loc[train_test['require_guest_profile_picture'] == 'f'] = 0
train_test['require_guest_profile_picture'].loc[train_test['require_guest_profile_picture'] == 't'] = 1

In [389]:
train_test['requires_license'].loc[train_test['requires_license'] == 'f'] = 0
train_test['requires_license'].loc[train_test['requires_license'] == 't'] = 1

In [394]:
train_test['reviews_per_month'].fillna(1, inplace=True)

In [399]:
temp = train.groupby('room_type')['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'].mean().reset_index()
temp.columns = ['room_type'] + [('mean_room_type_' + c) for c in temp.columns[1:]]
temp.head()
train_test = train_test.merge(temp, on='room_type', how='left')
train_test.drop('room_type', axis=1, inplace=True)

In [402]:
train_test.drop('scrape_id', axis=1, inplace=True)

In [404]:
train_test['security_deposit'].fillna(0, inplace=True)

In [406]:
train_test['security_deposit'] = train_test['security_deposit'].apply(cleaning_fee)

In [410]:
train_test.drop('smart_location', axis=1, inplace=True)

In [413]:
train_test['space'].fillna('none', inplace=True)

In [414]:
tqdm.pandas()
train_test['space_sentiment'] = train_test['space'].progress_apply(lambda x: sid.polarity_scores(x))


  1%|▏         | 1630/112024 [00:05<05:12, 353.03it/s]Exception KeyError: KeyError(<weakref at 000000000A4C4AE8; to 'tqdm' at 000000005E9FA748>,) in <object repr() failed> ignored

 10%|█         | 11731/112024 [00:31<03:13, 517.47it/s]


 28%|██▊       | 31588/112024 [01:02<01:46, 753.27it/s]


 52%|█████▏    | 58759/112024 [01:34<00:47, 1121.18it/s]


 70%|██████▉   | 78338/112024 [02:06<01:05, 517.68it/s]


 90%|████████▉ | 100307/112024 [02:37<00:13, 846.52it/s]


100%|██████████| 112024/112024 [02:51<00:00, 653.01it/s] 

In [415]:
train_test['space_compound'] = train_test['space_sentiment'].progress_apply(lambda x: x['compound'])
train_test['space_neg'] = train_test['space_sentiment'].progress_apply(lambda x: x['neg'])
train_test['space_neu'] = train_test['space_sentiment'].progress_apply(lambda x: x['neu'])
train_test['space_pos'] = train_test['space_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 405884.02it/s]


In [416]:
train_test.drop(['space', 'space_sentiment'], axis=1, inplace=True)

In [419]:
train_test.drop('square_feet', axis=1, inplace=True)

In [422]:
train_test.drop('state', axis=1, inplace=True)

In [424]:
train_test.drop('street', axis=1, inplace=True)

In [431]:
train_test['summary'].fillna('none', inplace=True)

In [432]:
train_test['summary_sentiment'] = train_test['summary'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [01:45<00:00, 1061.19it/s]


In [433]:
train_test['summary_compound'] = train_test['summary_sentiment'].progress_apply(lambda x: x['compound'])
train_test['summary_neg'] = train_test['summary_sentiment'].progress_apply(lambda x: x['neg'])
train_test['summary_neu'] = train_test['summary_sentiment'].progress_apply(lambda x: x['neu'])
train_test['summary_pos'] = train_test['summary_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 480789.62it/s]


In [434]:
train_test.drop(['summary', 'summary_sentiment'], axis=1, inplace=True)

In [459]:
train_test['transit'].fillna('none', inplace=True)

In [460]:
train_test['transit_sentiment'] = train_test['transit'].progress_apply(lambda x: sid.polarity_scores(x))

100%|██████████| 112024/112024 [01:28<00:00, 1270.93it/s]


In [461]:
train_test['transit_compound'] = train_test['transit_sentiment'].progress_apply(lambda x: x['compound'])
train_test['transit_neg'] = train_test['transit_sentiment'].progress_apply(lambda x: x['neg'])
train_test['transit_neu'] = train_test['transit_sentiment'].progress_apply(lambda x: x['neu'])
train_test['transit_pos'] = train_test['transit_sentiment'].progress_apply(lambda x: x['pos'])

100%|██████████| 112024/112024 [00:00<00:00, 374662.18it/s]


In [462]:
train_test.drop(['transit', 'transit_sentiment'], axis=1, inplace=True)

In [466]:
train_test.drop('weekly_price', axis=1, inplace=True)

In [435]:
train_test.drop('thumbnail_url', axis=1, inplace=True)

In [436]:
train_test.drop(['xl_picture_url', 'zipcode'], axis=1, inplace=True)

In [437]:
train_test.shape

(112024, 150)

In [438]:
train_test.head()

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,compound,neg,...,mean_room_type_review_scores_location,mean_room_type_review_scores_value,space_compound,space_neg,space_neu,space_pos,summary_compound,summary_neg,summary_neu,summary_pos
0,6,93,9.0,9.0,10.0,10.0,9.0,9.0,0.906302,0.013705,...,9.630505,9.479856,0.9829,0.033,0.797,0.171,0.9769,0.023,0.720,0.257
1,109,80,10.0,10.0,6.0,8.0,10.0,8.0,0.479400,0.014000,...,9.630505,9.479856,0.9854,0.000,0.826,0.174,0.8750,0.000,0.867,0.133
2,344,80,10.0,10.0,10.0,10.0,10.0,8.0,0.492650,0.000000,...,9.630505,9.479856,0.9577,0.000,0.501,0.499,0.9679,0.000,0.705,0.295
3,686,95,9.0,10.0,10.0,10.0,10.0,10.0,0.899540,0.010952,...,9.507798,9.500064,0.9861,0.000,0.816,0.184,0.4075,0.025,0.923,0.052
4,930,98,10.0,10.0,10.0,10.0,10.0,10.0,0.883906,0.019000,...,9.507798,9.500064,0.8852,0.019,0.874,0.108,0.9186,0.000,0.689,0.311


In [439]:
train_test.dtypes

listing_id                                          int64
review_scores_rating                                int64
review_scores_accuracy                            float64
review_scores_cleanliness                         float64
review_scores_checkin                             float64
review_scores_communication                       float64
review_scores_location                            float64
review_scores_value                               float64
compound                                          float64
neg                                               float64
neu                                               float64
pos                                               float64
accommodates                                        int64
availability_30                                     int64
availability_365                                    int64
availability_60                                     int64
availability_90                                     int64
bathrooms     

In [440]:
train.shape, test.shape

((70894, 104), (41130, 104))

In [468]:
train = train_test[:train.shape[0]]
test = train_test[train.shape[0]:]

In [469]:
train.shape, test.shape

((70894, 151), (41130, 151))

In [470]:
test.drop_duplicates('listing_id', inplace=True)

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [471]:
train.shape, test.shape

((70894, 151), (41129, 151))

In [472]:
train.head().T

,0,1,2,3,4
listing_id,6.000000,109.000000,344.000000,686.000000,930.000000
review_scores_rating,93.000000,80.000000,80.000000,95.000000,98.000000
review_scores_accuracy,9.000000,10.000000,10.000000,9.000000,10.000000
review_scores_cleanliness,9.000000,10.000000,10.000000,10.000000,10.000000
review_scores_checkin,10.000000,6.000000,10.000000,10.000000,10.000000
review_scores_communication,10.000000,8.000000,10.000000,10.000000,10.000000
review_scores_location,9.000000,10.000000,10.000000,10.000000,10.000000
review_scores_value,9.000000,8.000000,8.000000,10.000000,10.000000
compound,0.906302,0.479400,0.492650,0.899540,0.883906
neg,0.013705,0.014000,0.000000,0.010952,0.019000


In [473]:
to_drop = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'compound', 'neg', 'pos', 'neu', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'cleaning_fee', 'extra_people', 'guests_included', 'host_is_superhost', 'host_listings_count', 'host_response_rate', 'number_of_reviews', 'price', 'reviews_per_month', 'security_deposit']

In [474]:
train.drop(to_drop, axis=1, inplace=True)

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [475]:
test.drop(to_drop, axis=1, inplace=True)

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [3]:
train_sent_w2v_300_tfidf_500_loc_listings = pd.read_csv('train_sent_w2v_300_tfidf_500_loc_listing_countvec_500.csv')
test_sent_w2v_300_tfidf_500_loc_listings = pd.read_csv('test_sent_w2v_300_tfidf_500_loc_listing_countvec_500.csv')

In [477]:
train_sent_w2v_300_tfidf_500_loc_listings = train_sent_w2v_300_tfidf_500_loc_listings.merge(train, on='listing_id', how='left')
test_sent_w2v_300_tfidf_500_loc_listings = test_sent_w2v_300_tfidf_500_loc_listings.merge(test, on='listing_id', how='left')

In [478]:
train_sent_w2v_300_tfidf_500_loc_listings.shape, test_sent_w2v_300_tfidf_500_loc_listings.shape

((70894, 1470), (41129, 1463))

In [479]:
train_sent_w2v_300_tfidf_500_loc_listings.to_csv('train_sent_w2v_300_tfidf_500_loc_more_listing_countvec_500.csv', index=False)
test_sent_w2v_300_tfidf_500_loc_listings.to_csv('test_sent_w2v_300_tfidf_500_loc_more_listing_countvec_500.csv', index=False)

In [456]:
'transit' in train.columns

True

In [457]:
train['transit']

0        We are right around the corner from a bus stop...
1                                                      NaN
2        Short drive to subway and elevated trains runn...
3        Getting around Denver is easy from here withou...
4          Smart car rentals Bike rentals Train Buses Uber
5                                                      NaN
6        We are 1 block from the 1 or 5 bus routes that...
7        RTD bus stop is within two blocks.  The #1 bus...
8                                                      NaN
9        5 min walk to Capitol Metro Rail (train that t...
10       Subway 2.3.B.C. at 110th street around the cor...
11                                                     NaN
12                                   Car, Bike and Hike !!
13                                      Walk to bus stops!
14       You will be STEPS AWAY FROM THE L TRAIN. Hop o...
15                                                     NaN
16                                                     N